In [1]:
import pandas as pd
import gurobipy as gb
from gurobipy import *
import numpy as np

In [2]:
df = pd.read_csv('male_players (legacy).csv',encoding='latin1')
df['fifa_update_date'] = pd.to_datetime(df['fifa_update_date'])
df = df[df['nationality_name'] == 'Germany']
df = df[df['potential'] >= 79]
df = df[df['overall'] >= 75]

# remove injury player
df = df[df['player_id'] != 192448]
df = df[df['player_id'] != 188350]

# remove attend other match player
df = df[df['player_id'] != 256630]
df = df[df['player_id'] != 240709]

# Convert values in the 'preferred_foot' column to 0 for 'right' and 1 for 'left'
df['preferred_foot'] = df['preferred_foot'].apply(lambda x: 0 if x == 'Right' else 1)

# Choose 2021 
df = df[df['fifa_update_date'].dt.year == 2021]

/var/folders/3v/g2zdbcdx0czbtz7n9fzgd2180000gn/T/ipykernel_27359/3345018439.py:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('male_players (legacy).csv',encoding='latin1')


In [3]:
name = df["short_name"].tolist()
pn = df.iloc[:, -28:-1].columns
names = [p+' : '+ item for item in name for p in pn]
foot = df['preferred_foot'].tolist()
overall = df["overall"].tolist()
potential = df['potential'].tolist()
attack = df['attacking_finishing'].tolist()
defend_heading = df['defending_marking_awareness']+df['attacking_heading_accuracy']
defend_heading_list = defend_heading.tolist()
speed = df['movement_sprint_speed'].tolist()
cross = df['attacking_crossing'].tolist()
stamina = df["power_stamina"].tolist()
tackle = df['defending_standing_tackle'].tolist()
defawareness = df['defending_marking_awareness'].tolist()
control = df['skill_ball_control'].tolist()
positioning = df['mentality_positioning'].tolist()
num = len(name)
ability = df.iloc[:, -28:-1].values
for i in range(len(ability)):
    for j in range(len(ability[i])):
        ability[i][j] = eval(ability[i][j])
        if ability[i][j] not in range(0,100):
            print(ability[i][j])

In [4]:
model = gb.Model("2021 best23")
model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:
n = model.addVars(num,27,vtype = GRB.BINARY,name=names)
an = model.addVars(num,vtype = GRB.BINARY,name=name)
dh = model.addVars(num,vtype = GRB.BINARY,name=name)
potn = model.addVars(num,vtype = GRB.BINARY,name=name)

total = quicksum(n[i,j]*ability[i][j] for i in range(num) for j in range(27))
attacking = quicksum(an[i]*attack[i] for i in range(num))
dheading = quicksum(dh[i]*defend_heading_list[i] for i in range(num))
potentialing = quicksum(potn[i]*potential[i] for i in range(num))

model.setObjectiveN(attacking, index =0, priority = 3)
model.setObjectiveN(dheading, index =1, priority = 2)
model.setObjectiveN(potentialing/26, index =2, priority = 1)
model.setObjectiveN(total/26, index =3, priority = 0)

model.ModelSense = GRB.MAXIMIZE

model.addConstr(sum(an[i] for i in range(num))==3)
model.addConstr(sum(dh[i] for i in range(num))==2)
model.addConstr(sum(potn[i] for i in range(num))==26)

for i in range(num):
    model.addConstr(an[i] <= sum(n[i,j] for j in range(27)))
    model.addConstr(dh[i] <= sum(n[i,j] for j in range(27)))
    model.addConstr(potn[i] <= sum(n[i,j] for j in range(27)))

for i in range(num):
    model.addConstr(sum(n[i,j] for j in range(27))<=1)
model.addConstr(sum(n[i,j] for i in range(num) for j in range(27))==26)

# goal keeper equal to 3
model.addConstr(sum(n[i,26] for i in range(num))==3)

# forward at least 2
model.addConstr(sum(n[i,1] for i in range(num)) + sum(n[i,5] for i in range(num))>=3)

# lb and rb at least 1 each
model.addConstr(sum(n[i,21] for i in range(num)) >=2)

model.addConstr(sum(n[i,25] for i in range(num)) >=1)

# rcb and lcb at least 2 each
model.addConstr(sum(n[i,22] for i in range(num)) >=2)
model.addConstr(sum(n[i,24] for i in range(num)) >=2)

# lb must has preferred_foot at left and rb must has preferred_foot at right

model.addConstrs(n[i,21]<= foot[i] for i in range(num))
model.addConstrs(n[i,16]<= foot[i] for i in range(num))

model.addConstrs(n[i,25] + foot[i] <= 1 for i in range(num))
model.addConstrs(n[i,20] + foot[i] <= 1 for i in range(num))

# Each position no more than 3 players
model.addConstrs(sum(n[i,j] for i in range(num)) <=3 for j in range(27))

# Number of difference between left side and Right side should within 2 players
l = sum(n[i,0] +n[i,3]+n[i,4]+n[i,8]+n[i,11]+n[i,12]+n[i,16]+n[i,17]+n[i,21] for i in range(num))
r = sum(n[i,2] +n[i,6]+n[i,7]+n[i,10]+n[i,14]+n[i,15]+n[i,19]+n[i,20]+n[i,25] for i in range(num))
model.addConstr(l-r <=2)
model.addConstr(r-l <=2)

# Backfield at least 8 and no more than 10
back = sum(n[i,16] +n[i,20]+n[i,21]+n[i,22]+n[i,23]+n[i,24]+n[i,25]for i in range(num))
model.addConstr(back <=10)
model.addConstr(back >=8)

# Frontfield at least 3 and no more 6
f = sum(n[i,0] +n[i,1]+n[i,2]+n[i,3]+n[i,4]+n[i,5]+n[i,6]+n[i,7] for i in range(num))
model.addConstr(f <=6)
model.addConstr(f >=3)

Restricted license - for non-production use only - expires 2024-10-28
Set parameter MIPGap to value 1e-05


<gurobi.Constr *Awaiting Model Update*>

In [5]:
model.optimize()
print('overall rating is: ',model.getObjective(3).getValue())
for i in range(num):
        for j in range(27):
            if n[i, j].X > 0.5:  # Assuming a binary variable, check if it's set to 1
                print(n[i, j].VarName)

overall rating is:  84.26923076923077
gk : M. Neuer
cm : J. Kimmich
cm : T. Kroos
lam : T. Müller
rcm : L. Goretzka
lcb : M. Hummels
rcm : İ. Gündoğan
rm : S. Gnabry
rcb : M. Ginter
st : T. Werner
st : L. Sané
cam : K. Havertz
lcb : J. Boateng
gk : B. Leno
lam : K. Volland
rcb : A. Rüdiger
lb : R. Gosens
gk : K. Trapp
st : E. Can
rcb : N. Süle
rcm : F. Neuhaus
lb : M. Arnold
cam : J. Brandt
rb : L. Klostermann
cb : R. Koch
lam : J. Musiala


In [6]:
new_26_ability = []
new_26_name1 = []
new_foot = []
new_speed = []
new_cross = []
new_stamina = []
new_tackle = []
new_defawareness = []
new_control = []
new_positioning = []

for i in range(num):
    if sum(n[i,j].x for j in range(27)) ==1:
        new_26_ability.append(ability[i].tolist())
        new_26_name1.append(name[i])
        new_foot.append(foot[i])
        new_speed.append(speed[i])
        new_cross.append(cross[i])
        new_stamina.append(stamina[i])
        new_tackle.append(tackle[i])
        new_defawareness.append(defawareness[i])
        new_control.append(control[i])
        new_positioning.append(positioning[i])

num = len(new_26_name1)
new_26_name = [p+' : '+ item for item in new_26_name1 for p in pn]

In [7]:
# attack formation 433
model = gb.Model("2021 best11")
#model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:
n = model.addVars(num,27,vtype = GRB.BINARY,name=new_26_name)
sn = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)
cn = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)

total = quicksum(n[i,j]*new_26_ability[i][j] for i in range(num) for j in range(27))
speeding = quicksum(sn[i]*new_speed[i] for i in range(num))
crossing = quicksum(cn[i]*new_cross[i] for i in range(num))

model.setObjectiveN(speeding, index =2, priority = 0)
model.setObjectiveN(crossing, index =1, priority = 1)
model.setObjectiveN(total/11, index =0, priority = 2)
model.ModelSense = GRB.MAXIMIZE

model.addConstr(sum(sn[i] for i in range(num))==2)
model.addConstr(sum(cn[i] for i in range(num))==2)

for i in range(num):
    model.addConstr(sn[i] <= sum(n[i,j] for j in [3,7,11,15]))
    model.addConstr(cn[i] <= sum(n[i,j] for j in [21,25]))

for i in range(num):
    model.addConstr(sum(n[i,j] for j in range(27))<=1)
model.addConstr(sum(n[i,j] for i in range(num) for j in range(27))==11)

# goal keeper equal to 1
model.addConstr(sum(n[i,26] for i in range(num))==1)

# forward at least 1
model.addConstr(sum(n[i,1] for i in range(num)) + sum(n[i,5] for i in range(num))==1)

# lb,lwb and rb,rwb only 1 each
model.addConstr(sum(n[i,21] for i in range(num)) ==1)

model.addConstr(sum(n[i,25] for i in range(num)) ==1)

# rcb and lcb only 1 each
model.addConstr(sum(n[i,22] for i in range(num)) ==1)
model.addConstr(sum(n[i,24] for i in range(num)) ==1)

# lb must has preferred_foot at left and rb must has preferred_foot at right

model.addConstrs(n[i,21]<= new_foot[i] for i in range(num))
model.addConstrs(n[i,16]<= new_foot[i] for i in range(num))

model.addConstrs(n[i,25] + new_foot[i] <= 1 for i in range(num))
model.addConstrs(n[i,20] + new_foot[i] <= 1 for i in range(num))

# Each position only can be 1 players
model.addConstrs(sum(n[i,j] for i in range(num)) <=1 for j in range(27))

# Number of left side and Right side should be equal
l = sum(n[i,0] +n[i,3]+n[i,4]+n[i,8]+n[i,11]+n[i,12]+n[i,16]+n[i,17]+n[i,21] for i in range(num))
r = sum(n[i,2] +n[i,6]+n[i,7]+n[i,10]+n[i,14]+n[i,15]+n[i,19]+n[i,20]+n[i,25] for i in range(num))
model.addConstr(l==r)

rf = sum(n[i,6]+n[i,7]+n[i,15]+n[i,10] for i in range(num))
lf = sum(n[i,3]+n[i,4]+n[i,11]+n[i,8] for i in range(num))
model.addConstr(lf==rf)

# Midfield exactly 3
f = sum(n[i,9]+ n[i,12] +n[i,13]+n[i,14]+n[i,17]+n[i,18]+n[i,19] for i in range(num))
model.addConstr(f ==3)

# make sure not all locate at center
model.addConstr(sum(n[i,9]+ n[i,13] +n[i,18] for i in range(num)) ==1)


# Two wine
w = sum(n[i,3]+ n[i,7] +n[i,11]+n[i,15]for i in range(num))
model.addConstr(w ==2)

<gurobi.Constr *Awaiting Model Update*>

In [8]:
model.optimize()
#print('overall rating is: ',model.objVal)
for i in range(num):
        for j in range(27):
            if n[i, j].X > 0.5:  # Assuming a binary variable, check if it's set to 1
                print(n[i, j].VarName)
print('overall rating is: ',model.getObjective(0).getValue())

gk : M. Neuer
rb : J. Kimmich
lcm : T. Kroos
cam : T. Müller
rcm : L. Goretzka
rcb : M. Hummels
rm : S. Gnabry
lcb : M. Ginter
st : T. Werner
lm : L. Sané
lb : R. Gosens
overall rating is:  86.36363636363637


In [9]:
# 3 back formation 343
model = gb.Model("2021 3back best11")
#model.setParam('MIPGap', 0.00001)
# We ask Gurobi not to print too much on screen
model.Params.OutputFlag = 0

# Your code here:
n = model.addVars(num,27,vtype = GRB.BINARY,name=new_26_name)
cn = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)
stn = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)
tan = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)
den = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)
con = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)
pon = model.addVars(num,vtype = GRB.BINARY,name=new_26_name1)

total = quicksum(n[i,j]*new_26_ability[i][j] for i in range(num) for j in range(27))
staminaing = quicksum(stn[i]*new_stamina[i] for i in range(num))
tackling = quicksum(tan[i]*new_tackle[i] for i in range(num))
defawarenessing = quicksum(den[i]*new_defawareness[i] for i in range(num))
controling = quicksum(con[i]*new_control[i] for i in range(num))
positioning = quicksum(pon[i]*new_positioning[i] for i in range(num))
crossing = quicksum(cn[i]*new_cross[i] for i in range(num))


model.setObjectiveN(defawarenessing, index =6, priority = 0)
model.setObjectiveN(tackling, index =5, priority = 1)
model.setObjectiveN(positioning, index =4, priority = 2)
model.setObjectiveN(controling, index =3, priority = 3)
model.setObjectiveN(staminaing, index =2, priority = 4)
model.setObjectiveN(crossing, index =1, priority = 5)
model.setObjectiveN(total/11, index =0, priority = 6)
model.ModelSense = GRB.MAXIMIZE

model.addConstr(sum(stn[i] for i in range(num))==2)
model.addConstr(sum(tan[i] for i in range(num))==1)
model.addConstr(sum(den[i] for i in range(num))==2)
model.addConstr(sum(con[i] for i in range(num))==2)
model.addConstr(sum(pon[i] for i in range(num))==3)


for i in range(num):
    model.addConstr(stn[i] <= sum(n[i,j] for j in [16,20]))
    model.addConstr(tan[i] <= n[i,23])
    model.addConstr(den[i] <= sum(n[i,j] for j in [22,24]))
    model.addConstr(con[i] <= sum(n[i,j] for j in [12,13,14,17,18,19]))
    model.addConstr(pon[i] <= sum(n[i,j] for j in range(8)))
    

for i in range(num):
    model.addConstr(sum(n[i,j] for j in range(27))<=1)
model.addConstr(sum(n[i,j] for i in range(num) for j in range(27))==11)

# goal keeper equal to 1
model.addConstr(sum(n[i,26] for i in range(num))==1)

# forward at least 1
model.addConstr(sum(n[i,1] for i in range(num)) + sum(n[i,5] for i in range(num))>=1)

# lb,lwb and rb,rwb only 1 each
model.addConstr(sum(n[i,21]+n[i,16] for i in range(num)) ==1)

model.addConstr(sum(n[i,25]+n[i,20] for i in range(num)) ==1)

# one CB
model.addConstr(sum(n[i,23] for i in range(num)) == 1)

# one lwb and rwb
model.addConstr(sum(n[i,20] for i in range(num)) == 1)
model.addConstr(sum(n[i,16] for i in range(num)) == 1)

# rcb and lcb only 1 each
model.addConstr(sum(n[i,22] for i in range(num)) ==1)
model.addConstr(sum(n[i,24] for i in range(num)) ==1)

# lb must has preferred_foot at left and rb must has preferred_foot at right

model.addConstrs(n[i,21]<= new_foot[i] for i in range(num))
model.addConstrs(n[i,16]<= new_foot[i] for i in range(num))

model.addConstrs(n[i,25] + new_foot[i] <= 1 for i in range(num))
model.addConstrs(n[i,20] + new_foot[i] <= 1 for i in range(num))

# Each position only can be 1 players
model.addConstrs(sum(n[i,j] for i in range(num)) <=1 for j in range(27))

# Number of left side and Right side should be equal
l = sum(n[i,0] +n[i,3]+n[i,4]+n[i,8]+n[i,11]+n[i,12]+n[i,16]+n[i,17]+n[i,21] for i in range(num))
r = sum(n[i,2] +n[i,6]+n[i,7]+n[i,10]+n[i,14]+n[i,15]+n[i,19]+n[i,20]+n[i,25] for i in range(num))
model.addConstr(l==r)

rf = sum(n[i,6]+n[i,7]+n[i,15]+n[i,10] for i in range(num))
lf = sum(n[i,3]+n[i,4]+n[i,11]+n[i,8] for i in range(num))
model.addConstr(lf==rf)

# Midfield only 2
f = sum(n[i,12] +n[i,13]+n[i,14]+n[i,17]+n[i,18]+n[i,19] for i in range(num))
model.addConstr(f ==2)

# can't only locate center
model.addConstr(sum(n[i,13] +n[i,18] for i in range(num)) ==0)

# A. Rüdiger perfer lcb
model.addConstr(n[15,24] ==0)

<gurobi.Constr *Awaiting Model Update*>

In [10]:
model.optimize()
#print('overall rating is: ',model.objVal)
for i in range(num):
        for j in range(27):
            if n[i, j].X > 0.5:  # Assuming a binary variable, check if it's set to 1
                print(n[i, j].VarName)
print('overall rating is: ',model.getObjective(0).getValue())

gk : M. Neuer
rwb : J. Kimmich
lcm : T. Kroos
ls : T. Müller
rcm : L. Goretzka
cb : M. Hummels
rcb : M. Ginter
st : T. Werner
rs : L. Sané
lcb : A. Rüdiger
lwb : R. Gosens
overall rating is:  86.27272727272728


In [11]:
# real team squad and overall rating
dfreal = pd.read_csv('male_players (legacy).csv',encoding='latin1')
dfreal['fifa_update_date'] = pd.to_datetime(dfreal['fifa_update_date'])
dfreal = dfreal[dfreal['fifa_update_date'].dt.year == 2021]
real_23player_list = [167495,205452,210665,207862,178603, 212622, 235790,182521, 200610, 206113, 212188,192563,
                     210324, 256790,212190,222331,234943,209658,222492,223697,186942,188943,208333,236532,189596,209846]
real_df = dfreal[dfreal['player_id'].isin(real_23player_list)]
print(np.mean(real_df['overall']))
print(real_df[['player_id','short_name','overall']])

/var/folders/3v/g2zdbcdx0czbtz7n9fzgd2180000gn/T/ipykernel_27359/1603381870.py:2: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  dfreal = pd.read_csv('male_players (legacy).csv',encoding='latin1')


83.11538461538461
        player_id      short_name  overall
123818     167495        M. Neuer       90
123830     212622      J. Kimmich       89
123835     182521        T. Kroos       88
123846     189596      T. MÃ¼ller       87
123851     209658     L. Goretzka       87
123860     178603      M. Hummels       86
123889     186942  Ä°. GÃ¼ndoÄan       85
123900     206113       S. Gnabry       85
123930     207862       M. Ginter       84
123937     212188       T. Werner       84
123943     222492        L. SanÃ©       84
123949     235790      K. Havertz       84
123965     192563         B. Leno       83
123973     200610      K. Volland       83
123981     205452     A. RÃ¼diger       83
123995     223697       R. Gosens       83
124022     188943        K. Trapp       82
124046     208333          E. Can       82
124055     212190        N. SÃ¼le       82
124086     234943      F. Neuhaus       82
124275     222331  L. Klostermann       80
124390     209846      C. GÃ¼nter   

In [12]:
# real 343
print((90+85+86+84+88+83+85+88+86+86+83)/11)

85.81818181818181
